In [48]:
import buckeye
from buckeye.utterance import words_to_utterances

import pathlib2
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt
from datetime import datetime
import librosa
from avgn.utils.json import NoIndent, NoIndentEncoder

In [24]:
from avgn.utils.paths import DATA_DIR, ensure_dir
from avgn.utils.audio import int16_to_float32

In [25]:
import tempfile
from scipy.io import wavfile

In [26]:
# create a unique datetime identifier for the files output by this notebook
DT_ID = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
DT_ID

'2019-10-08_21-32-02'

In [27]:
DATASET_ID = 'buckeye'

In [28]:
BUCKEYE_DIR = pathlib2.Path('/mnt/cube/Datasets/buckeye/')

In [29]:
speaker_list = list(BUCKEYE_DIR.glob('*.zip'))
len(speaker_list), speaker_list[0]

(40, PosixPath('/mnt/cube/Datasets/buckeye/s36.zip'))

In [30]:
fd, TEMPFILE = tempfile.mkstemp()

### create JSON and save wav

In [53]:
for speaker_loc in tqdm(speaker_list):
    speaker = buckeye.Speaker.from_zip(speaker_loc, load_wavs=True)
    for track in tqdm([track for track in speaker], leave=False, desc='track'):

        # output locations
        wav_out = (
            DATA_DIR / "processed" / DATASET_ID / DT_ID / "WAV" / (track.name + ".WAV")
        )
        json_out = (
            DATA_DIR
            / "processed"
            / DATASET_ID
            / DT_ID
            / "JSON"
            / (track.name + ".JSON")
        )

        track.clip_wav(TEMPFILE, 0, track.words[-1].end)
        fs, data = wavfile.read(TEMPFILE)

        # save wav file
        ensure_dir(wav_out)
        librosa.output.write_wav(wav_out, y=int16_to_float32(data), sr=fs, norm=True)

        # wav general information
        json_dict = {}
        json_dict["species"] = "Homo sapien"
        json_dict["age"] = speaker.age
        json_dict["sex"] = speaker.sex
        json_dict["common_name"] = "Human"
        json_dict["samplerate_hz"] = fs
        json_dict["length_s"] = len(data) / fs
        json_dict["wav_loc"] = wav_out.as_posix()
        json_dict["indvs"] = {
            speaker.name: {
                "words": {
                    "start_times": [],
                    "end_times": [],
                    "labels": [],
                    "utterance_number": [],
                    "PoS": [],
                    "position_in_utterance": [],
                },
                "phones": {
                    "start_times": [],
                    "end_times": [],
                    "labels": [],
                    "word": [],
                    "utterance_number": [],
                    "position_in_utterance": [],
                    "word_number": [],
                    "position_in_word": [],
                },
            }
        }
        
        for ui, utterance in tqdm(enumerate(words_to_utterances(track.words)), leave=False, desc="utt"):
            pi = 0
            for wi, word in enumerate(utterance):
                if word.misaligned is False:
                    if hasattr(word, "phonemic"):  # if this is not a pause, etc
                        json_dict["indvs"][speaker.name]["words"]['start_times'].append(word.beg)
                        json_dict["indvs"][speaker.name]["words"]['end_times'].append(word.end)
                        json_dict["indvs"][speaker.name]["words"]['labels'].append(word.orthography)
                        json_dict["indvs"][speaker.name]["words"]['utterance_number'].append(ui)
                        json_dict["indvs"][speaker.name]["words"]['PoS'].append(word.pos)
                        json_dict["indvs"][speaker.name]["words"]['position_in_utterance'].append(wi)

                        for phonei, phone in enumerate(word.phones):
                            json_dict["indvs"][speaker.name]["phones"]['start_times'].append(phone.beg)
                            json_dict["indvs"][speaker.name]["phones"]['end_times'].append(phone.end)
                            json_dict["indvs"][speaker.name]["phones"]['labels'].append(phone.seg)
                            json_dict["indvs"][speaker.name]["phones"]['word'].append(word.orthography)
                            json_dict["indvs"][speaker.name]["phones"]['utterance_number'].append(pi)
                            json_dict["indvs"][speaker.name]["phones"]['position_in_utterance'].append(wi)
                            json_dict["indvs"][speaker.name]["phones"]['word_number'].append(wi)
                            json_dict["indvs"][speaker.name]["phones"]['position_in_word'].append(phonei)
                            pi+=1

        for key, val in json_dict["indvs"][speaker.name]["words"].items():
            json_dict["indvs"][speaker.name]["words"][key] = NoIndent(val)
        
        for key, val in json_dict["indvs"][speaker.name]["phones"].items():
            json_dict["indvs"][speaker.name]["phones"][key] = NoIndent(val)
            
        json_txt = json.dumps(json_dict, cls=NoIndentEncoder, indent=2)
        # save json
        ensure_dir(json_out.as_posix())
        print(json_txt, file=open(json_out.as_posix(), "w"))
        